# Case Study - The Current - Part 2

* The Current is an alternative radio station
* We will pull information about the play list.

# Step 0 - Insert current progress

Copy over all the relevant code from part 1 of the lab.

http://www.thecurrent.org/playlist/2014-01-01/01

In [47]:
!pip install composable
!pip install composablesoup
!pip install composable --upgrade
!pip install composablesoup --upgrade
from composablesoup import find, find_all, get_text, has_attr
from composable.sequence import slice, head
from composable.strict import map, filter
from composable.string import replace, split
from composable import from_toolz as tlz
from composable import pipeable

Requirement already up-to-date: composable in /home/ki6241pu/.pyenv/versions/anaconda3-2020.02/lib/python3.7/site-packages (0.2.5)
Requirement already up-to-date: composablesoup in /home/ki6241pu/.pyenv/versions/anaconda3-2020.02/lib/python3.7/site-packages (0.2.4)


In [2]:
import requests
from bs4 import BeautifulSoup
s = requests.Session()
r = s.get('https://www.thecurrent.org/playlist/2014-01-01/01')
the_current = BeautifulSoup(r.content, "html.parser")

# Pull off the song start time

1. Inspect the element
    1. This one is tricky
    2. Time tag does not have a tag, but
    3. The surrounding div does have a class
2. Identify the html tag and class
3. Use `find_all` to make a list of all relevant tags
4. Pull off an example case
5. Write a function that extracts the start time.
6. Write a single pipe to extract the start time.
7. Confirm you have the right number of times.
8. Package your code in a function called `get_start_time`

In [7]:
(the_current
 >> find_all('div', attrs = {'class':'songTime'})
)

[<div class="two columns songTime">
 <a href="#song226645">
 <time>  1:59 </time>
 </a>
 </div>,
 <div class="two columns songTime">
 <a href="#song196069">
 <time>  1:54 </time>
 </a>
 </div>,
 <div class="two columns songTime">
 <a href="#song229900">
 <time>  1:51 </time>
 </a>
 </div>,
 <div class="two columns songTime">
 <a href="#song235779">
 <time>  1:46 </time>
 </a>
 </div>,
 <div class="two columns songTime">
 <a href="#song132616">
 <time>  1:44 </time>
 </a>
 </div>,
 <div class="two columns songTime">
 <a href="#song224268">
 <time>  1:38 </time>
 </a>
 </div>,
 <div class="two columns songTime">
 <a href="#song236492">
 <time>  1:34 </time>
 </a>
 </div>,
 <div class="two columns songTime">
 <a href="#song237794">
 <time>  1:31 </time>
 </a>
 </div>,
 <div class="two columns songTime">
 <a href="#song234211">
 <time>  1:27 </time>
 </a>
 </div>,
 <div class="two columns songTime">
 <a href="#song235959">
 <time>  1:23 </time>
 </a>
 </div>,
 <div class="two columns songT

In [8]:
(the_current
 >> find_all('div', attrs = {'class':'songTime'})
 >> map(get_text)
)

['\n\n  1:59 \n\n',
 '\n\n  1:54 \n\n',
 '\n\n  1:51 \n\n',
 '\n\n  1:46 \n\n',
 '\n\n  1:44 \n\n',
 '\n\n  1:38 \n\n',
 '\n\n  1:34 \n\n',
 '\n\n  1:31 \n\n',
 '\n\n  1:27 \n\n',
 '\n\n  1:23 \n\n',
 '\n\n  1:19 \n\n',
 '\n\n  1:13 \n\n',
 '\n\n  1:09 \n\n',
 '\n\n  1:05 \n\n',
 '\n\n  1:03 \n\n',
 '\n\n  1:01 \n\n']

In [3]:
strip = pipeable(lambda s: s.strip())

In [4]:
get_start_time = pipeable( lambda soup: soup
                         >> find_all('div', attrs = {'class':'two columns songTime'})
                         >> map(get_text)
                         >> map(strip)
                        )
the_current >> get_start_time

['1:59',
 '1:54',
 '1:51',
 '1:46',
 '1:44',
 '1:38',
 '1:34',
 '1:31',
 '1:27',
 '1:23',
 '1:19',
 '1:13',
 '1:09',
 '1:05',
 '1:03',
 '1:01']

# Pull address of the album art image address

Follow a similar process to pull off the web address of the album cover image. 


In [114]:
(the_current
 >> find_all('img', attrs = {'class':'album-art'})
)

[<img alt="We The Common" class="album-art lazyload" data-src="https://albumart.publicradio.org/mb/e2/e2749c25-c2b6-493e-a2bb-10898152bd2d_5158.jpg" src="" title="Thao and The Get Down Stay Down - We The Common"/>,
 <img alt="default album cover image" class="album-art" src="/assets/album-cover-default-32217dc68a771f3a44aa2b7a640cf91133b61bd1f2ae68c9ddb00055e9a8ac1d.png"/>,
 <img alt="default album cover image" class="album-art" src="/assets/album-cover-default-32217dc68a771f3a44aa2b7a640cf91133b61bd1f2ae68c9ddb00055e9a8ac1d.png"/>,
 <img alt="Wildewoman" class="album-art lazyload" data-src="https://albumart.publicradio.org/mb/5e/5e5c8b95-d04c-432f-8cd2-c1c8d99e6e5a_3556.jpg" src="" title="Lucius - Wildewoman"/>,
 <img alt="Frosting on the Beater" class="album-art lazyload" data-src="https://albumart.publicradio.org/mb/48/48445b64-d965-369a-af3c-8193de389fd8_3ff4.jpg" src="" title="The Posies - Frosting on the Beater"/>,
 <img alt="default album cover image" class="album-art" src="/ass

In [153]:
test = pipeable(lambda L: [tlz.get('data-src') for i in L])

In [106]:
import re
give_default = pipeable(lambda l: ["/assets/album-cover-default-32217dc....png" for i in l if i == ""] )

In [159]:
get_img_address = pipeable( lambda soup: soup
                          >> find_all('img', attrs = {'class':'album-art'})
                          >> filter(has_attr('data-src'))
                          #>> give_default
                          >> map(tlz.get('data-src'))
                           #>> test
                          )
the_current >> get_img_address

['https://albumart.publicradio.org/mb/e2/e2749c25-c2b6-493e-a2bb-10898152bd2d_5158.jpg',
 'https://albumart.publicradio.org/mb/5e/5e5c8b95-d04c-432f-8cd2-c1c8d99e6e5a_3556.jpg',
 'https://albumart.publicradio.org/mb/48/48445b64-d965-369a-af3c-8193de389fd8_3ff4.jpg',
 'https://albumart.publicradio.org/mb/e9/e999c049-c65b-4c5e-ad12-5596998679c7_92f9.jpg',
 'https://albumart.publicradio.org/mb/d6/d62320e2-20c4-4589-aa76-2f8ac28447dd_e03b.jpg',
 'https://albumart.publicradio.org/mb/02/028b8602-3bde-495a-a7da-15594fc4f786_351a.jpg',
 'https://albumart.publicradio.org/mb/c9/c92f73ee-527f-42ed-a556-fd615941e214_78f0.jpg',
 'https://albumart.publicradio.org/mb/24/24084807-5d23-423e-b1f3-5e9fd874e240_6ccd.jpg',
 'https://albumart.publicradio.org/mb/c2/c20be759-d767-4a7c-96c5-7a870ebc3a30_7f7d.jpg',
 'https://albumart.publicradio.org/mb/37/37f48931-e5e6-488f-a531-ad2db311158d_7446.jpg',
 'https://albumart.publicradio.org/mb/1a/1aa41b19-5a72-341b-bd91-4cf61d1dab6b_8e05.jpg']

In [95]:
help(tlz.get)

Help on pipeable in module toolz.itertoolz:

get(ind='__no__default__', seq='__no__default__', default='__no__default__')
    Get element in a sequence or dict
    
    Provides standard indexing
    
    >>> get(1, 'ABC')       # Same as 'ABC'[1]
    'B'
    
    Pass a list to get multiple values
    
    >>> get([1, 2], 'ABC')  # ('ABC'[1], 'ABC'[2])
    ('B', 'C')
    
    Works on any value that supports indexing/getitem
    For example here we see that it works with dictionaries
    
    >>> phonebook = {'Alice':  '555-1234',
    ...              'Bob':    '555-5678',
    ...              'Charlie':'555-9999'}
    >>> get('Alice', phonebook)
    '555-1234'
    
    >>> get(['Alice', 'Bob'], phonebook)
    ('555-1234', '555-5678')
    
    Provide a default for missing values
    
    >>> get(['Alice', 'Dennis'], phonebook, None)
    ('555-1234', None)
    
    See Also:
        pluck



# Putting it all together

* Make a function for each of the previous steps
* Make an overall function
    * input is a soup
    * output is a list of lists

**Hint:** You should use `zip` to put all the information together.

In [9]:
get_period = pipeable( lambda soup: soup
                         >> find('span', attrs = {'class':'hour-header'})
                         >> get_text
                         >> strip
                         >> split(' ')
                         >> tlz.get(1)
                         )
get_dj = pipeable(lambda soup: soup
                    >> find('h5', attrs = {'class':'currentDj'})
                    >> get_text
                    >> split(': ')
                    >> tlz.get(1)
                 )
get_weekday = pipeable(lambda soup: soup
                    >> find('a', attrs = {'class':'start-picker'})
                    >> get_text
                    >> split(',')
                    >> tlz.get(0)
                 )
get_title = pipeable(lambda soup: soup
                     >> find_all('h5', attrs = {'class':'title'})
                     >> map(get_text)
                 )
get_artist = pipeable(lambda soup: soup
                     >> find_all('h5', attrs = {'class':'artist'})
                     >> map(get_text)
                 )
get_start_time = pipeable( lambda soup: soup
                         >> find_all('div', attrs = {'class':'two columns songTime'})
                         >> map(get_text)
                         >> map(strip)
                        )
get_img_address = pipeable( lambda soup: soup
                          >> find_all('img', attrs = {'class':"album-art"})
                          >> filter(has_attr('data-src'))
                          >> map(tlz.get('data-src'))
                          )

In [10]:
from composable.strict import zipOnto
#from composable.list import to_list
combine = (the_current
           >> zip(get_period, get_dj)
           >> to_list
           >> map(comma_join)
          )
combine

TypeError: zip argument #1 must support iteration

In [86]:
combine = zip(get_period, get_dj)

TypeError: zip argument #1 must support iteration

In [87]:
import itertools

In [90]:
all_info = lambda soup: zip(itertools.repeat(soup >> get_period), itertools.repeat(soup >> get_dj), itertools.repeat(soup >> get_weekday), soup >> get_title, soup >> get_artist, soup >> get_start_time)
tuple(all_info(the_current))

(('am',
  'Jade',
  'Wednesday',
  'Holy Roller',
  'Thao and The Get Down Stay Down',
  '1:59'),
 ('am', 'Jade', 'Wednesday', 'Kingdom of Rust', 'Doves', '1:54'),
 ('am', 'Jade', 'Wednesday', 'Black Dog', 'Frankie Lee', '1:51'),
 ('am', 'Jade', 'Wednesday', 'Turn It Around', 'Lucius', '1:46'),
 ('am', 'Jade', 'Wednesday', 'Flavor of the Month', 'The Posies', '1:44'),
 ('am', 'Jade', 'Wednesday', 'Potential Wife', 'Strange Names', '1:38'),
 ('am', 'Jade', 'Wednesday', '24 Hours', 'Sky Ferreira', '1:34'),
 ('am',
  'Jade',
  'Wednesday',
  "Who's Gonna Shoe Your Pretty Little Feet?",
  'Billie Joe and Norah',
  '1:31'),
 ('am',
  'Jade',
  'Wednesday',
  'Marigold',
  'J. Roddy Walston and The Business',
  '1:27'),
 ('am', 'Jade', 'Wednesday', 'High Road', 'Cults', '1:23'),
 ('am',
  'Jade',
  'Wednesday',
  'The Vampyre Of Time and Memory',
  'Queens of the Stone Age',
  '1:19'),
 ('am', 'Jade', 'Wednesday', 'Valerie Plame', 'The Decemberists', '1:13'),
 ('am', 'Jade', 'Wednesday', 'Mo